In [1]:
import numpy as np
import math
import tensorflow as tf

In [2]:
def trapezoidIntegral(vals, ds):
    pass1 = tf.reduce_sum(vals, axis=0)
    pass2 = tf.reduce_sum(vals[1:-1], axis=0)
    return ds/2.0*(pass1+pass2)
def TFconv(image, filt, b):
    return tf.nn.convolution(input=image, filters=filt, padding="SAME") + b

In [3]:
def TFuSoln(raw_u0, ds, inFilt, pFilt, b):
    bzero = tf.zeros([inFilt.shape[4]])
    u0 = TFconv(raw_u0, inFilt[0], bzero)
    #for loop begins, h=0
    x0 = tf.nn.leaky_relu(u0, alpha=0.1)
    zInt0 = TFconv(x0, pFilt[0,0], b[0] / ((0+1)*ds))
    z0 = trapezoidIntegral(zInt0, ds)
    u1 = z0 + TFconv(raw_u0, inFilt[1], bzero)
    #h=1
    x1 = tf.nn.leaky_relu(u1, alpha=0.1)
    zInt1_0 = TFconv(x0, pFilt[0,1], b[1] / ((1+1)*ds)) #s=0
    zInt1_1 = TFconv(x1, pFilt[1,1], b[1] / ((1+1)*ds)) #s=1
    #zInt1 = zInt1_0 + zInt1_1
    zInt1 = tf.stack([zInt1_0, zInt1_1])
    z1 = trapezoidIntegral(zInt1, ds)
    u2 = z1 + TFconv(raw_u0, inFilt[2], bzero)
    #h=2
    x2 = tf.nn.leaky_relu(u2, alpha=0.1)#
    zInt2_0 = TFconv(x0, pFilt[0,2], b[2] / ((2+1)*ds)) #s=0
    zInt2_1 = TFconv(x1, pFilt[1,2], b[2] / ((2+1)*ds)) #s=1
    zInt2_2 = TFconv(x2, pFilt[2,2], b[2] / ((2+1)*ds)) #s=2
    #zInt2 = zInt2_0 + zInt2_1 + zInt2_2
    zInt2 = tf.stack([zInt2_0, zInt2_1, zInt2_2])
    z2 = trapezoidIntegral(zInt2, ds)#
    u3 = z2 + TFconv(raw_u0, inFilt[3], bzero)
    #h=3
    x3 = tf.nn.leaky_relu(u3, alpha=0.1)
    zInt3_0 = TFconv(x0, pFilt[0,3], b[3] / ((3+1)*ds)) #s=0
    zInt3_1 = TFconv(x1, pFilt[1,3], b[3] / ((3+1)*ds)) #s=1
    zInt3_2 = TFconv(x2, pFilt[2,3], b[3] / ((3+1)*ds)) #s=2
    zInt3_3 = TFconv(x3, pFilt[3,3], b[3] / ((3+1)*ds)) #s=3
    #zInt3 = zInt3_0 + zInt3_1 + zInt3_2 + zInt3_3
    zInt3 = tf.stack([zInt3_0, zInt3_1, zInt3_2, zInt3_3])
    z3 = trapezoidIntegral(zInt3, ds)
    u4 = z3 + TFconv(raw_u0, inFilt[4], bzero)
    #end loop
    x4 = tf.nn.leaky_relu(u4, alpha=0.1)
    
    #print("z with h=0", z0)
    #print("z with h=1", z1)
    #print("z with h=2", z2)
    #print("z with h=3", z3)
    
    x = tf.stack([x0,x1,x2,x3,x4])
    u = tf.stack([u0,u1,u2,u3,u4])
    
    return x,u

In [8]:
t0 = 4
t1 = t0+1
dt = 1.0/t0
input_shape = (1,5,5)
filter_shape = (1,1,3,3)



actWt = [.1, 1] #leakyRelU: first number is slope left of origin, second number is slope right of origin

rng = np.random.default_rng(42)
uIn = rng.random(input_shape, dtype=np.float32)
iFilt = rng.random((t1,*filter_shape), dtype=np.float32)
paramFilt = rng.random((t1,t1,*filter_shape), dtype=np.float32)
biasVec = rng.random((t1,1), dtype=np.float32)

#xOut, uOut = uSoln(uIn, dt, iFilt, paramFilt, biasVec)
#print(xOut)

In [5]:
TFuIn = tf.expand_dims(tf.transpose(tf.convert_to_tensor(uIn, dtype=tf.float32), perm=[1,2,0]), 0)
TFiFilt = tf.transpose(tf.convert_to_tensor(iFilt, dtype=tf.float32), perm=[0,3,4,1,2])
TFpFilt = tf.transpose(tf.convert_to_tensor(paramFilt, dtype=tf.float32), perm=[0,1,4,5,2,3])
TFbiasVec = tf.convert_to_tensor(biasVec, dtype=tf.float32)

TFxOut, TFuOut = TFuSoln(TFuIn, dt, TFiFilt, TFpFilt, TFbiasVec)
print(tf.transpose(TFxOut, perm=[4,0,1,2,3]))

tf.Tensor(
[[[[[ 0.5149827   1.2764959   1.1055958   0.9759438   0.48584086]
    [ 1.6816992   2.3455856   2.2812831   2.1016786   0.84754264]
    [ 1.4462036   2.5093234   2.2971082   2.0258262   1.1332246 ]
    [ 1.8460461   2.300061    2.8805401   2.5757613   1.7191321 ]
    [ 1.1274791   1.3376107   1.773575    1.6535478   1.5739689 ]]]


  [[[ 1.8183213   2.137969    2.526722    1.8751593   1.0164145 ]
    [ 2.8927186   4.2703075   3.8896542   3.7959993   1.8713158 ]
    [ 2.7698338   4.4808173   3.9417782   4.136777    2.268245  ]
    [ 2.913106    3.7311845   4.747446    4.4243755   3.077777  ]
    [ 1.4768031   2.3763497   3.1060374   2.426774    2.4768248 ]]]


  [[[ 2.5133073   4.035907    3.9692717   3.5524368   2.1501262 ]
    [ 4.1414165   6.1417      6.551713    5.583663    3.6007633 ]
    [ 4.226528    7.424518    6.782687    6.510955    4.781315  ]
    [ 4.5307074   6.1021676   6.6800737   6.9026937   5.1220803 ]
    [ 2.7774005   3.6048331   4.821697    4.891281    3.5